# 多输入多输出通道

实现一下多输入通道互相关运算

In [1]:
import torch
from d2l import torch as d2l
'''
zip(X,K)：按通道维度将输入和卷积核一一配对
x₀（第0通道输入）配k₀（第0通道卷积核）
x₁（第1通道输入）配k₁（第1通道卷积核）
'''
def corr2d_multi_in(X, K):
    # 对X和K的每个通道分别计算互相关，然后将结果累加
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

验证互相关运算的输出，<br>![本地路径](./img/多输入通道.png)

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

计算多个通道的输出的互相关函数

In [3]:
'''
nn.Conv2d要求权重是4D的：[输出通道, 输入通道,高,宽]
torch.stack把多个3D核→1个4D核张量
每个输出通道对应一个3D卷积核，这样就可以同时计算多个输出特征图
corr2d_multi_in_out：实现多输入多输出通道卷积
torch.stack：创建多个卷积核，每个核提取不同特征
'''
def corr2d_multi_in_out(X, K):
    '''
    对K中的每个卷积核k，计算多通道卷积，然后将所有结果堆叠起来
    for k in K：遍历多个卷积核（K现在是4D张量）
    corr2d_multi_in(X, k)：每个卷积核与多通道输入X计算，得到一张2D特征图
    torch.stack(..., 0)：将所有2D特征图在第0维（新增维度）堆叠，形成3D输出（多通道）
    '''
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
# 将三个卷积核沿第0维堆叠
K = torch.stack((K, # 原始卷积核（3D）
                 K + 1, # 每个元素加1
                 K + 2), # 每个元素加2
                 0)
K.shape

torch.Size([3, 2, 2, 2])

In [4]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

1x1卷积

In [5]:
def corr2d_multi_in_out_1x1(X, K):
    # 获取输入形状: (输入通道, 高, 宽)
    c_i, h, w = X.shape
    # 获取输出通道数（卷积核个数）
    c_o = K.shape[0]
    '''
    将输入从3D重塑为2D: (c_i, h * w)
    (3, 2, 2)[通道, 高, 宽]→(3, 4):
    [ 1,  2,  3,  4], ← 通道0在所有4个位置的值
    [ 5,  6,  7,  8], ← 通道1在所有4个位置的值
    [ 9, 10, 11, 12]  ← 通道2在所有4个位置的值
    通道在所有4个位置的值：每一列代表一个空间位置（共4个位置），每一行是一个通道。
    每个空间位置(h,w)展开成一列，共h*w列，每列代表一个空间位置，有c_i个通道的值。
    '''
    X = X.reshape((c_i, h * w))
    '''
    将卷积核从4D重塑为2D: (c_o, c_i)
    目的：(2, 3)→去掉1×1的空洞维度，暴露其权重矩阵本质
    [0.1, 0.2, 0.3], ← 输出通道0的权重（对3个输入通道）
    [0.4, 0.5, 0.6]  ← 输出通道1的权重（对3个输入通道）
    每一行是一个输出通道的权重向量，每一列对应一个输入通道的权重。
    1×1卷积核就是 c_o 个 c_i 维向量，每个向量负责生成一个输出通道。
    '''
    K = K.reshape((c_o, c_i))
    # 矩阵乘法: (c_o, c_i) × (c_i, h*w) → (c_o, h*w)
    # 这等价于在每个空间位置上做全连接
    # # (2, 3) × (3, 4) → (2, 4)
    Y = torch.matmul(K, X)
    # 将结果重塑回3D: (输出通道, 高, 宽) (2, 2, 2)
    return Y.reshape((c_o, h, w))
# 随机输入: 3通道, 3x3
X = torch.normal(0, 1, (3, 3, 3))
# 1x1卷积核: 2个输出通道, 3个输入通道, 核大小1x1
K = torch.normal(0, 1, (2, 3, 1, 1))
# 方法1: 11x1卷积：矩阵乘法
Y1 = corr2d_multi_in_out_1x1(X, K)
# 方法2: 标准多通道卷积实现
Y2 = corr2d_multi_in_out(X, K)
# 断言两者结果误差极小（数值计算精度范围内）
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6